In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import RidgeCV, LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold, cross_val_score
from tqdm.notebook import tqdm
import json
from rca import process_categorical, best_logistic_solver, make_binary_scoring, make_multiclass_scoring, checker
import pickle

## Loading Data

In [2]:
rca = pd.read_csv('../../data/results/rca.csv').dropna()
meta = pd.read_csv('../../data/psychNorms/psychNorms_metadata_processed.csv', index_col=0)
norms = pd.read_csv('../../data/psychNorms/psychNorms_processed.zip', index_col=0, compression='zip', low_memory=False)

# Adding norm_cat to rca
rca['norm_cat'] = (
    rca['norm'].apply(lambda norm: meta.loc[norm]['category'])
    .replace({'_': ' '}, regex=True)
)

rca

,embed,embed_type,norm,train_n,test_n,p,r2_mean,r2_sd,check,norm_cat
0,CBOW_GoogleNews,text,frequency_lund,28012,7003,300,0.522118,0.008398,pass,frequency
1,CBOW_GoogleNews,text,frequency_kucera,19285,4822,300,0.500425,0.009710,pass,frequency
2,CBOW_GoogleNews,text,frequency_subtlexus,28636,7159,300,0.537265,0.009814,pass,frequency
3,CBOW_GoogleNews,text,frequency_subtlexuk,29316,7330,300,0.545643,0.008416,pass,frequency
4,CBOW_GoogleNews,text,frequency_blog_gimenes,31876,7969,300,0.523700,0.008448,pass,frequency
...,...,...,...,...,...,...,...,...,...,...
6979,THINGS,behavior,imageability_vanarsdall,376,95,49,0.074112,0.090266,pass,imageability
6980,THINGS,behavior,familiarity_fear,173,44,49,0.151637,0.158767,pass,familiarity
6981,THINGS,behavior,aoa_fear,173,44,49,0.015012,0.129597,pass,age of acquisition
6982,THINGS,behavior,imageability_fear,173,44,49,-0.023230,0.101781,pass,imageability


In [3]:
embed_avgs = (
    rca[['embed', 'norm_cat', 'r2_mean']]
    .groupby(['embed', 'norm_cat']).median(numeric_only=True) # median is used to mitigate outliers within norm_cats
    .groupby('embed').mean()
    .rename(columns={'r2_mean': 'r2_avg'})
)
embed_avgs

,r2_avg
embed,
CBOW_GoogleNews,0.447376
EEG_speech,-0.065708
EEG_text,-0.047123
GloVe_CommonCrawl,0.313229
GloVe_Twitter,0.296789
GloVe_Wikipedia,0.299761
LexVec_CommonCrawl,0.290600
PPMI_SVD_EAT,0.296205
PPMI_SVD_SWOW,0.398496


In [4]:
# Adding embed types
with open('../../data/embed_to_dtype.json', 'r') as f:
    embed_to_type = json.load(f)
embed_avgs['type'] = embed_avgs.index.map(embed_to_type)

# Finding top 2 text 
text_name_1, text_name_2 = (
    embed_avgs.query('type == "text"').sort_values('r2_avg', ascending=False).head(2).index.tolist()
)
text_name_1, text_name_2

('CBOW_GoogleNews', 'fastText_CommonCrawl')

In [5]:
# Finding top behavior
behavior_name = (
    embed_avgs.query('type == "behavior"').sort_values('r2_avg', ascending=False).head(1).index[0]
)
behavior_name

'PPMI_SVD_SWOW'

In [6]:
# Loading embeds
text_1 = pd.read_csv(f'../../data/embeds/{text_name_1}.csv', index_col=0)
text_2 = pd.read_csv(f'../../data/embeds/{text_name_2}.csv', index_col=0)
behavior = pd.read_csv(f'../../data/embeds/{behavior_name}.csv', index_col=0)

with open('../../data/brain_behav_union.pkl', 'rb') as f:
    brain_behav_union = pickle.load(f)

# Subsdetting to brain and behavior union and aligning vocabs
intersect = sorted(list(set.intersection(
    set(text_1.index), set(text_2.index), set(behavior.index), set(brain_behav_union)
)))
text_1, text_2, behavior = text_1.loc[intersect], text_2.loc[intersect], behavior.loc[intersect]

# Standardizing
standardize = lambda df: (df - df.mean()) / df.std()
text_1, text_2, behavior = standardize(text_1), standardize(text_2), standardize(behavior)

# Ensembling for comparison
embeds = {
    behavior_name: behavior,
    text_name_1: text_1, 
    text_name_2: text_2,
    text_name_1 + '&' + text_name_2: pd.concat([text_1, text_2], axis=1),
    text_name_1 + '&' + behavior_name: pd.concat([text_1, behavior], axis=1),
    text_name_2 + '&' + behavior_name: pd.concat([text_2, behavior], axis=1)
}

# Fixing column names
for embed_name, embed in embeds.items():
    embed.columns = list(range(embed.shape[1]))
    embeds[embed_name] = embed

{name: embed.shape for name, embed in embeds.items()}

{'PPMI_SVD_SWOW': (11723, 300),
 'CBOW_GoogleNews': (11723, 300),
 'fastText_CommonCrawl': (11723, 300),
 'CBOW_GoogleNews&fastText_CommonCrawl': (11723, 600),
 'CBOW_GoogleNews&PPMI_SVD_SWOW': (11723, 600),
 'fastText_CommonCrawl&PPMI_SVD_SWOW': (11723, 600)}

## Cross Validation

In [7]:
# Changing associated_embed to more usable format
meta['associated_embed'] = meta['associated_embed'].str.split(' ')

# Ridge
min_ord, max_ord = -5, 5
alphas = np.logspace(
    min_ord, max_ord, max_ord - min_ord + 1
)
ridge = RidgeCV(alphas=alphas)

# Logistic hyperparameters
Cs = 1 / alphas
inner_cv = 5
penalty = 'l2'

# Scorers
binary_scoring = make_binary_scoring()
multiclass_scoring = make_multiclass_scoring()
continuous_scoring = 'r2'

# outer_cv setting 
outer_cv, n_jobs = 5, 6

solo_embed_names = [text_name_1, text_name_2, behavior_name] # For checking data leakage in checker

# RCA
rca = []
for norm_name in tqdm(norms.columns):
    print(f'{norm_name}:')
    y = norms[norm_name].dropna()
    
    to_print = []
    for embed_name, embed in embeds.items():
        
        # Aligning embed with norm
        X, y = embed.align(y, axis='index', join='inner', copy=True)
        
        # Checking norm dtype 
        norm_dtype = meta.loc[norm_name, 'type']
        
        # Solvers, scoring, estimators ir categorical or continuous
        if norm_dtype in ['binary', 'multiclass']: # categorical
            X, y = process_categorical(outer_cv, inner_cv, X, y)
            
            # may have switched form multi to bin after processing
            norm_dtype = 'binary' if len(y.unique()) == 2 else 'multiclass'
            
            # Cross validation settings for logistic regression
            solver = best_logistic_solver(y, norm_dtype)
            
            # Defining logistic regression 
            estimator = LogisticRegressionCV(
                Cs=Cs, penalty=penalty, cv=StratifiedKFold(inner_cv), solver=solver
            )
            scoring = binary_scoring if norm_dtype == 'binary' else multiclass_scoring
        else: # continuous
            estimator, scoring = ridge, continuous_scoring
            
        # Cross validation
        associated_embed = meta.loc[norm_name, 'associated_embed']
        check = checker(solo_embed_names, y, norm_dtype, associated_embed, outer_cv)
        if check == 'pass':
            r2s = cross_val_score(estimator, X, y, cv=outer_cv, scoring=scoring, n_jobs=n_jobs) # stratification is automatically used for classification
        else:
            r2s = pd.Series([np.nan] * outer_cv)
            
        # Saving
        train_n = int(((outer_cv - 1) / outer_cv) * len(y))
        for i, r2 in enumerate(r2s):
            rca.append([embed_name, norm_name, train_n, i + 1, r2, check])
            
        # Printing
        to_print.append([embed_name, r2s.mean(), r2s.std(), check])
    to_print = pd.DataFrame(to_print, columns=['embed', 'r2_mean', 'r2_std', 'check'])
    print(to_print.sort_values('r2_mean', ascending=False).head(10).reset_index(drop=True))
    print('--------------------------------')
 
 
rca = pd.DataFrame(
    rca, columns=[
        'embed', 'norm', 'train_n', 'fold', 'r2', 'check']
)
rca.to_csv('../../data/results/rca_ensemb.csv', index=False)
rca

  0%|          | 0/291 [00:00<?, ?it/s]

frequency_lund:
                                  embed   r2_mean    r2_std check
0    fastText_CommonCrawl&PPMI_SVD_SWOW  0.829188  0.010604  pass
1  CBOW_GoogleNews&fastText_CommonCrawl  0.816586  0.013173  pass
2                  fastText_CommonCrawl  0.801812  0.012247  pass
3         CBOW_GoogleNews&PPMI_SVD_SWOW  0.723285  0.013075  pass
4                       CBOW_GoogleNews  0.665994  0.011748  pass
5                         PPMI_SVD_SWOW  0.515268  0.026448  pass
--------------------------------
frequency_kucera:
                                  embed   r2_mean    r2_std check
0    fastText_CommonCrawl&PPMI_SVD_SWOW  0.727461  0.025132  pass
1  CBOW_GoogleNews&fastText_CommonCrawl  0.712266  0.028553  pass
2                  fastText_CommonCrawl  0.690077  0.029289  pass
3         CBOW_GoogleNews&PPMI_SVD_SWOW  0.659806  0.021880  pass
4                       CBOW_GoogleNews  0.602879  0.021317  pass
5                         PPMI_SVD_SWOW  0.474919  0.043541  pass
---------

/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model

                                  embed   r2_mean    r2_std check
0                  fastText_CommonCrawl  0.771914  0.006750  pass
1    fastText_CommonCrawl&PPMI_SVD_SWOW  0.771180  0.011473  pass
2  CBOW_GoogleNews&fastText_CommonCrawl  0.765757  0.007569  pass
3         CBOW_GoogleNews&PPMI_SVD_SWOW  0.709304  0.011728  pass
4                       CBOW_GoogleNews  0.699886  0.009189  pass
5                         PPMI_SVD_SWOW  0.479968  0.009043  pass
--------------------------------
pos_vanheuven:


/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model

...

/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model

                                  embed   r2_mean    r2_std check
0    fastText_CommonCrawl&PPMI_SVD_SWOW  0.238479  0.021820  pass
1                         PPMI_SVD_SWOW  0.238320  0.018761  pass
2         CBOW_GoogleNews&PPMI_SVD_SWOW  0.233106  0.021992  pass
3  CBOW_GoogleNews&fastText_CommonCrawl  0.223877  0.012682  pass
4                  fastText_CommonCrawl  0.210079  0.026419  pass
5                       CBOW_GoogleNews  0.168470  0.047998  pass
--------------------------------
semantic_diversity_hoffman:
                                  embed   r2_mean    r2_std check
0  CBOW_GoogleNews&fastText_CommonCrawl  0.736929  0.009970  pass
1    fastText_CommonCrawl&PPMI_SVD_SWOW  0.726702  0.009477  pass
2                  fastText_CommonCrawl  0.723172  0.009614  pass
3         CBOW_GoogleNews&PPMI_SVD_SWOW  0.700762  0.009484  pass
4                       CBOW_GoogleNews  0.686559  0.011982  pass
5                         PPMI_SVD_SWOW  0.492860  0.021252  pass
---------------

/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/datapool-1/homepoint/zhussain/anaconda3/envs/psychProbing/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
 

                                  embed   r2_mean    r2_std check
0  CBOW_GoogleNews&fastText_CommonCrawl  0.576792  0.070503  pass
1                  fastText_CommonCrawl  0.565636  0.138068  pass
2                       CBOW_GoogleNews  0.551300  0.074584  pass
3    fastText_CommonCrawl&PPMI_SVD_SWOW  0.535739  0.220545  pass
4         CBOW_GoogleNews&PPMI_SVD_SWOW  0.473083  0.178319  pass
5                         PPMI_SVD_SWOW  0.463057  0.290280  pass
--------------------------------
imagery_toronto:
                                  embed   r2_mean    r2_std check
0    fastText_CommonCrawl&PPMI_SVD_SWOW  0.669332  0.056579  pass
1  CBOW_GoogleNews&fastText_CommonCrawl  0.649006  0.039851  pass
2         CBOW_GoogleNews&PPMI_SVD_SWOW  0.640592  0.052086  pass
3                  fastText_CommonCrawl  0.620359  0.057510  pass
4                       CBOW_GoogleNews  0.579855  0.039124  pass
5                         PPMI_SVD_SWOW  0.578119  0.078404  pass
--------------------------

,embed,norm,train_n,fold,r2,check
0,PPMI_SVD_SWOW,frequency_lund,9040,1,0.478024,pass
1,PPMI_SVD_SWOW,frequency_lund,9040,2,0.507218,pass
2,PPMI_SVD_SWOW,frequency_lund,9040,3,0.549852,pass
3,PPMI_SVD_SWOW,frequency_lund,9040,4,0.500720,pass
4,PPMI_SVD_SWOW,frequency_lund,9040,5,0.540526,pass
...,...,...,...,...,...,...
8725,fastText_CommonCrawl&PPMI_SVD_SWOW,sensory_experience_juhasz2013,3715,1,0.476002,pass
8726,fastText_CommonCrawl&PPMI_SVD_SWOW,sensory_experience_juhasz2013,3715,2,0.419785,pass
8727,fastText_CommonCrawl&PPMI_SVD_SWOW,sensory_experience_juhasz2013,3715,3,0.488512,pass
8728,fastText_CommonCrawl&PPMI_SVD_SWOW,sensory_experience_juhasz2013,3715,4,0.409050,pass
